<a href="https://colab.research.google.com/github/mrkiranilla/AWS-oct/blob/main/SatelliteImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -----------------------------------------
# STEP 1 — Upload & unzip
# -----------------------------------------

from google.colab import files
import zipfile
import os

# Upload your zip
uploaded = files.upload()

# Extract to 'data'
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# -----------------------------------------
# STEP 2 — Robust auto-detect dataset root
# -----------------------------------------

def find_deepest_dataset_root(base_dir='data'):
    """
    Recursively find the deepest folder that contains class subfolders.
    """
    for root, dirs, files in os.walk(base_dir):
        # if this folder has subfolders and NO images directly inside, it's probably the parent
        if len(dirs) >= 2 and not any(f.lower().endswith(('.jpg', '.png', '.jpeg')) for f in files):
            return root
    # fallback: maybe there's only one folder
    for root, dirs, files in os.walk(base_dir):
        if len(dirs) >= 1:
            return os.path.join(root, dirs[0])
    raise Exception("❌ Could not find dataset root with class folders.")

dataset_path = find_deepest_dataset_root()
print(f"✅ Dataset path auto-detected: {dataset_path}")

# -----------------------------------------
# STEP 3 — Load dataset
# -----------------------------------------

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(dataset_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

print(f"✅ Found {len(dataset.classes)} classes: {dataset.classes}")

# -----------------------------------------
# STEP 4 — Define CNN
# -----------------------------------------

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN(num_classes=len(dataset.classes)).to(device)
print("✅ Model created")

# -----------------------------------------
# STEP 5 — Train
# -----------------------------------------

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = 100 * correct / len(train_dataset)
    print(f"✅ Epoch {epoch+1}/{num_epochs} | Loss: {running_loss:.3f} | Accuracy: {accuracy:.2f}%")

# -----------------------------------------
# STEP 6 — Save & Download model
# -----------------------------------------

torch.save(model.state_dict(), 'model.pth')
print("✅ Model saved as model.pth")

from google.colab import files
files.download('model.pth')


Saving archive.zip to archive.zip
✅ Dataset path auto-detected: data/data
✅ Found 4 classes: ['cloudy', 'desert', 'green_area', 'water']
✅ Model created
✅ Epoch 1/5 | Loss: 68.122 | Accuracy: 78.29%
✅ Epoch 2/5 | Loss: 41.685 | Accuracy: 88.03%
✅ Epoch 3/5 | Loss: 40.100 | Accuracy: 88.43%
✅ Epoch 4/5 | Loss: 40.072 | Accuracy: 88.12%
✅ Epoch 5/5 | Loss: 37.624 | Accuracy: 88.92%
✅ Model saved as model.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>